<img src="https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png" width="300" align="right"/>

# ICOS Carbon Portal Python Library

# Example: Load data based on a sparql query


## Documentation
Full documentation for the library on the [project page](https://icos-carbon-portal.github.io/pylib/), how to install and wheel on [pypi.org](https://pypi.org/project/icoscp/"), source available on [github](https://github.com/ICOS-Carbon-Portal/pylib)

## Import the library and create a dobj representation

In [ ]:
from icoscp.cpb.dobj import Dobj
from icoscp.sparql.runsparql import RunSparql

<img src="img/sparql.png" width="400"  style="float:right">

## You can copy your own sparql query, based on your search criteria 

- Go to https://data.icos-cp.eu  and find datasets you want
- press the icon in the middle of the screen (see image to the right),  to copy your sparql query
- come back here and create the variable `query` 

For the following example, we have searched for:<br>
ICOS, EcoSystem data, Deciduous Broadleaf Forests, Fluxdata Preview, Level 2<br>
https://data.icos-cp.eu/portal/#%7B%22filterCategories%22%3A%7B%22project%22%3A%5B%22icos%22%5D%2C%22theme%22%3A%5B%22ecosystem%22%5D%2C%22ecosystem%22%3A%5B%22igbp_DBF%22%5D%2C%22level%22%3A%5B2%5D%2C%22type%22%3A%5B%22etcFluxnetProduct%22%5D%7D%7D

with the goal to plot NEE over time

In [ ]:
query = '''
prefix cpmeta: <http://meta.icos-cp.eu/ontologies/cpmeta/>
prefix prov: <http://www.w3.org/ns/prov#>
select ?dobj ?spec ?fileName ?size ?submTime ?timeStart ?timeEnd
where {
	VALUES ?spec {<http://meta.icos-cp.eu/resources/cpmeta/etcFluxnetProduct>}
	?dobj cpmeta:hasObjectSpec ?spec .
	VALUES ?station {<http://meta.icos-cp.eu/resources/stations/ES_BE-Lcr> <http://meta.icos-cp.eu/resources/stations/ES_DE-HoH> <http://meta.icos-cp.eu/resources/stations/ES_FR-Fon> <http://meta.icos-cp.eu/resources/stations/ES_DE-Hai>}
			?dobj cpmeta:wasAcquiredBy/prov:wasAssociatedWith ?station .
	?dobj cpmeta:hasSizeInBytes ?size .
?dobj cpmeta:hasName ?fileName .
?dobj cpmeta:wasSubmittedBy/prov:endedAtTime ?submTime .
?dobj cpmeta:hasStartTime | (cpmeta:wasAcquiredBy / prov:startedAtTime) ?timeStart .
?dobj cpmeta:hasEndTime | (cpmeta:wasAcquiredBy / prov:endedAtTime) ?timeEnd .
	FILTER NOT EXISTS {[] cpmeta:isNextVersionOf ?dobj}
}
'''

In [ ]:
result = RunSparql(query, 'pandas')   # look at the documentation for different outputformats...
result.run()
result.data()

In [ ]:
result.data()['dobj']

In [ ]:
dobj_list = []
for d in result.data()['dobj']:
    dobj_list.append(Dobj(d))
    

In [ ]:
dobj_list

In [ ]:
for o in dobj_list:
    display(o.data.head())

In [ ]:
# bokeh for plotting the data
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot, column, row
from bokeh.io import output_notebook
from bokeh.models import Div
output_notebook()

In [ ]:
figure_list = []
title = Div(text='<h2>Net Ecosystem Exchange</h2>')

start_date = '01-01-2020'
end_date = '30-06-2020'

for o in dobj_list:
    mask = (o.data['TIMESTAMP'] > start_date) & (o.data['TIMESTAMP'] <= end_date)
    data = o.data[mask]  # filter by date
    
    # create a figure
    fig = figure(plot_width=300, plot_height=300, title=o.station, x_axis_type='datetime')
    # plot data    
    fig.circle(data['TIMESTAMP'], data['NEE_VUT_REF'], size=1, color="navy", alpha=0.3)
    
    #append to list
    figure_list.append(fig)
    
#link x & y axis for all figures
for i in range(1,len(figure_list)):    
    figure_list[i].x_range=figure_list[0].x_range
    figure_list[i].y_range=figure_list[0].y_range
    
grid = gridplot([figure_list])
show(column(title, grid))